In [1]:
import pandas as pd
import xgboost as xgb
from config import *
from sklearn.preprocessing import MinMaxScaler

train1 = pd.read_csv(train_file_path1)
train2 = pd.read_csv(train_file_path2)
test = pd.read_csv(test_file_path)

train = pd.concat([train1,train2],axis=0)

train1_x = train1.drop(['label'],axis=1)
train1_y = train1.label

train2_x = train2.drop(['label'],axis=1)
train2_y = train2.label

train_x = train.drop(['label'],axis=1)
train_y = train.label


test_x = test.drop(['User_id','Coupon_id','Merchant_id','Date_received'],axis=1)
test_preds = test[['User_id','Coupon_id','Date_received']]

print train_x.shape,test_x.shape

train = xgb.DMatrix(train_x,label=train_y)
train1 = xgb.DMatrix(train1_x,label=train1_y)
train2 = xgb.DMatrix(train2_x,label=train2_y)
test = xgb.DMatrix(test_x)

(393427, 38) (112803, 38)


In [2]:
params={'booster':'gbtree',
	    'objective': 'rank:pairwise',
	    'eval_metric':'auc',
	    'gamma':0.1,
	    'min_child_weight':1.1,
	    'max_depth':5,
	    'lambda':10,
	    'subsample':0.7,
	    'colsample_bytree':0.7,
	    'colsample_bylevel':0.7,
	    'eta': 0.01,
	    'tree_method':'exact',
	    'seed':0,
	    'nthread':12
	    }

In [6]:
model = xgb.Booster()
model.load_model("0002.model")

In [ ]:
#train on dataset1, evaluate on dataset2
# watchlist = [(train1,'train'),(train2,'val')]
watchlist = [(train,'train')]
# model = xgb.train(params,train,num_boost_round=300,evals=watchlist) #,early_stopping_rounds=300,xgb_model=model)
model = xgb.train(params,train,num_boost_round=7000,evals=watchlist,early_stopping_rounds=300)#,xgb_model=model)

# watchlist = [(train,'train')]
# model = xgb.train(params,train,num_boost_round=3500,evals=watchlist)


[0]	train-auc:0.807914
Will train until train-auc hasn't improved in 300 rounds.
[1]	train-auc:0.816055
[2]	train-auc:0.817883
[3]	train-auc:0.822917
[4]	train-auc:0.827317
[5]	train-auc:0.828242
[6]	train-auc:0.829824
[7]	train-auc:0.829844
[8]	train-auc:0.831231
[9]	train-auc:0.831093
[10]	train-auc:0.831679
[11]	train-auc:0.832544
[12]	train-auc:0.833229
[13]	train-auc:0.833162
[14]	train-auc:0.833815
[15]	train-auc:0.833545
[16]	train-auc:0.833773
[17]	train-auc:0.833856
[18]	train-auc:0.834379
[19]	train-auc:0.834736
[20]	train-auc:0.835341
[21]	train-auc:0.835668
[22]	train-auc:0.835389
[23]	train-auc:0.835454
[24]	train-auc:0.83532
[25]	train-auc:0.835446
[26]	train-auc:0.835589
[27]	train-auc:0.83558
[28]	train-auc:0.835645
[29]	train-auc:0.835919
[30]	train-auc:0.836008
[31]	train-auc:0.836153
[32]	train-auc:0.836465
[33]	train-auc:0.836289
[34]	train-auc:0.836486
[35]	train-auc:0.836402
[36]	train-auc:0.836447
[37]	train-auc:0.836491
[38]	train-auc:0.836704
[39]	train-auc:0.8

In [22]:
model.best_score

0.911596

In [23]:
model.save_model('0002.model')

In [19]:
model2 = xgb.Booster()
model2.load_model("0002.model")

In [20]:
# watchlist2 = [(train1,'train'),(train2,'val')]
# model2 = xgb.train(params,train1,num_boost_round=300,evals=watchlist2,early_stopping_rounds=300,xgb_model=model2)

[0]	train-auc:0.911549	val-auc:0.906487
Multiple eval metrics have been passed: 'val-auc' will be used for early stopping.

Will train until val-auc hasn't improved in 300 rounds.
[1]	train-auc:0.911597	val-auc:0.906472
[2]	train-auc:0.911703	val-auc:0.906465
[3]	train-auc:0.911751	val-auc:0.906467
[4]	train-auc:0.911817	val-auc:0.906462
[5]	train-auc:0.911918	val-auc:0.906437
[6]	train-auc:0.911978	val-auc:0.90643
[7]	train-auc:0.912058	val-auc:0.906435
[8]	train-auc:0.912131	val-auc:0.906424
[9]	train-auc:0.912222	val-auc:0.906412
[10]	train-auc:0.912303	val-auc:0.906401
[11]	train-auc:0.912366	val-auc:0.906365
[12]	train-auc:0.912418	val-auc:0.906349
[13]	train-auc:0.912488	val-auc:0.906321
[14]	train-auc:0.912621	val-auc:0.906281
[15]	train-auc:0.912668	val-auc:0.906262
[16]	train-auc:0.912735	val-auc:0.906246
[17]	train-auc:0.91278	val-auc:0.90621
[18]	train-auc:0.912834	val-auc:0.906191
[19]	train-auc:0.912895	val-auc:0.906164
[20]	train-auc:0.912952	val-auc:0.90614
[21]	train-au

KeyboardInterrupt: 

In [24]:
#save feature score
feature_score = model.get_fscore()
feature_score = sorted(feature_score.items(), key=lambda x:x[1],reverse=True)
fs = []
for (key,value) in feature_score:
    fs.append("{0},{1}\n".format(key,value))
fs

['day_of_month,5069\n',
 'merchant_sales_count,4060\n',
 'Distance,3644\n',
 'avg_merchant_date_datereceived_gap,3202\n',
 'user_this_month_all_coupon_count,3131\n',
 'merchant_mean_distance,2645\n',
 'coupon_uesd_sales_rate,2625\n',
 'merchant_coupon_count,2549\n',
 'merchant_coupon_used_rate,2484\n',
 'day_of_week,2280\n',
 'discount_man,2278\n',
 'Discount_rate,2248\n',
 'merchant_coupon_notused_count,2244\n',
 'user_received_count,1989\n',
 'user_notuse_coupon_count,1973\n',
 'user_merchant_count,1748\n',
 'max_merchant_date_datereceived_gap,1728\n',
 'user_merchant_received_count,1699\n',
 'merchant_coupon_used_count,1574\n',
 'avg_user_date_datereceived_gap,1557\n',
 'discount_jian,1444\n',
 'max_user_date_datereceived_gap,1141\n',
 'user_this_day_all_coupon_count,1111\n',
 'min_user_date_datereceived_gap,1041\n',
 'min_merchant_date_datereceived_gap,821\n',
 'user_buy_use_coupon_rate,777\n',
 'user_merchant_buy_use_coupon_rate,719\n',
 'merchant_max_distance,653\n',
 'user_coupo

In [33]:
#predict test set
test_preds.loc[:,'Probability'] = model.predict(test)
test_preds.Probability = MinMaxScaler().fit_transform(test_preds.Probability)
test_preds.sort_values(by=['Coupon_id','Probability'],inplace=True)
test_preds.to_csv("preds.csv",index=None)




/usr/local/Cellar/anaconda/lib/python2.7/site-packages/sklearn/preprocessing/data.py:324: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/usr/local/Cellar/anaconda/lib/python2.7/site-packages/sklearn/preprocessing/data.py:359: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)


In [31]:
train_x

,Distance,day_of_week,day_of_month,is_weekend,user_this_month_all_coupon_count,user_this_day_all_coupon_count,user_received_count,user_notuse_coupon_count,user_buy_use_coupon_count,user_min_distance,...,merchant_coupon_used_rate,coupon_uesd_sales_rate,Discount_rate,discount_man,discount_jian,is_man_jian,user_merchant_received_count,user_merchant_buy_use_coupon_count,user_merchant_buy_use_coupon_rate,user_merchant_buy_use_coupon_of_user_buy_use_coupon_rate
0,0.0,5,29,0,1,1,NaN,NaN,NaN,NaN,...,0.013809,0.092635,0.900000,200.0,20.0,1,NaN,NaN,NaN,NaN
1,10.0,4,21,0,1,1,NaN,NaN,NaN,NaN,...,0.003626,0.099111,0.850000,200.0,30.0,1,NaN,NaN,NaN,NaN
2,10.0,2,26,0,1,1,3.0,3.0,0.0,NaN,...,0.013809,0.092635,0.900000,200.0,20.0,1,1.0,0.0,0.0,0.0
3,NaN,5,29,0,1,1,1.0,1.0,0.0,NaN,...,0.009752,0.047459,0.833333,30.0,5.0,1,NaN,NaN,NaN,NaN
4,9.0,2,26,0,1,1,15.0,12.0,3.0,0.0,...,0.003626,0.099111,0.850000,200.0,30.0,1,1.0,0.0,0.0,0.0
5,0.0,7,17,1,1,1,NaN,NaN,NaN,NaN,...,0.003626,0.099111,0.850000,200.0,30.0,1,NaN,NaN,NaN,NaN
6,NaN,5,29,0,1,1,NaN,NaN,NaN,NaN,...,0.000000,0.000000,0.500000,100.0,50.0,1,NaN,NaN,NaN,NaN
7,NaN,5,15,0,1,1,NaN,NaN,NaN,NaN,...,0.041801,0.140194,0.833333,30.0,5.0,1,NaN,NaN,NaN,NaN
8,0.0,2,10,0,1,1,6.0,6.0,0.0,NaN,...,0.118413,0.075988,0.750000,20.0,5.0,1,1.0,0.0,0.0,0.0
9,10.0,5,29,0,1,1,NaN,NaN,NaN,NaN,...,0.013809,0.092635,0.900000,200.0,20.0,1,NaN,NaN,NaN,NaN
